In [25]:
import pyvo as vo
from pyvo import dal, registry
from astropy import units as u
from astropy.coordinates import SkyCoord
import polars as pl

# Jupyter dropdown widget
from ipywidgets import widgets


In [3]:
# Define the cone search parameters
ra = 10.68479
dec = 41.26906
radius = 0.1

# Convert to SkyCoord
coord = SkyCoord(ra, dec, unit='deg')


In [8]:
coord.galactic

<SkyCoord (Galactic): (l, b) in deg
    (121.17440713, -21.57300123)>

In [12]:

# Get a list of all the services which support cone searches and covers the area we are interested in
services = registry.search(servicetype='conesearch', spatial=coord)

print(len(services))

1945


In [16]:
# Show the top 5 from the pyvo registry table
services.to_table()[:5]

ivoid,res_type,short_name,res_title,content_level,res_description,reference_url,creator_seq,content_type,source_format,source_value,region_of_regard,waveband,access_urls,standard_ids,intf_types,intf_roles
,,,,,,,,,,,deg,,,,,
object,object,object,object,object,object,object,object,object,object,object,float32,object,object,object,object,object
ivo://astron.nl/hetdex/lotss-dr1/cone,vs:catalogservice,LoTSS-DR1 Cone Search,LoTSS-DR1 Cross-Matched Source Catalogue,,"This service queries the catalog of radio sources from the LOFAR\nTwo-metre Sky Survey First Data Release (LoTSS-DR1) that have been\ncross-matched with an optical or infrared counterpart. This data\nrelease contains images and catalogs that characterise the\nlow-frequency radio emission in the region of the HETDEX Spring Field\n(right ascension 10h45m00s to15h30m00s and declination 45◦00′00′′ to\n57◦00′00′′). A total of 325,694 radio sources are detected in a region\ncovering 424 square degrees. The maps have a median sensitivity of 71\nuJy/beam and a resolution of 6 arcsec. Optical counterparts for 71% of\nthe radio sources have been identified and where possible photometric\nredshifts for these sources have been derived.",https://vo.astron.nl/hetdex/lotss-dr1/cone/info,LOFAR Surveys KSP,,,,--,radio,https://vo.astron.nl/hetdex/lotss-dr1/cone/scs.xml?,ivo://ivoa.net/std/conesearch,vs:paramhttp,std
ivo://astron.nl/mvf/q/cone,vs:catalogservice,MSSS Cone Search,MSSS Verification Field Sources,,"This service queries the unified source catalogue of radio sources\nfrom the Multifrequency Snapshot Sky Survey (MSSS) Verification Field.\nThis survey is the first major observing program to be carried out\nwith LOFAR during its ongoing commissioning phase. This service\nqueries the unified source catalogue database for the MSSS survey. The\nVerification Field is a region of 100 square degrees centered at\n(15h,69◦).",https://vo.astron.nl/mvf/q/cone/info,"Heald, G.;Pizzo, R.;et al.",,,http://msss.astron.nl,--,radio,https://vo.astron.nl/mvf/q/cone/scs.xml?,ivo://ivoa.net/std/conesearch,vs:paramhttp,std
ivo://au.csiro/casda/scs,vs:catalogservice,CSIRO ASKAP Cone,CSIRO ASKAP Science Data Archive Cone Search Service,university#research,Cone search service for querying catalogues from ASKAP radio astronomy observations,http://www.atnf.csiro.au/observers/data/index.html,CSIRO,other,,,--,radio,https://casda.csiro.au/casda_vo_tools/scs/spectral_line_absorption?:::py VO sep:::https://casda.csiro.au/casda_vo_tools/scs/polarisation_component?:::py VO sep:::https://casda.csiro.au/casda_vo_tools/scs/spectral_line_emission?:::py VO sep:::https://casda.csiro.au/casda_vo_tools/scs/continuum_component?:::py VO sep:::https://casda.csiro.au/casda_vo_tools/scs/continuum_island?:::py VO sep:::https://casda.csiro.au/casda_vo_tools/scs/racs_dr1_sources_galacticregion_v2021_08_v01?:::py VO sep:::https://casda.csiro.au/casda_vo_tools/scs/racs_dr1_sources_galacticcut_v2021_08_v01?:::py VO sep:::https://casda.csiro.au/casda_vo_tools/scs/obscore?,ivo://ivoa.net/std/conesearch:::py VO sep:::ivo://ivoa.net/std/conesearch:::py VO sep:::ivo://ivoa.net/std/conesearch:::py VO sep:::ivo://ivoa.net/std/conesearch:::py VO sep:::ivo://ivoa.net/std/conesearch:::py VO sep:::ivo://ivoa.net/std/conesearch:::py VO sep:::ivo://ivoa.net/std/conesearch:::py VO sep:::ivo://ivoa.net/std/conesearch,vs:paramhttp:::py VO sep:::vs:paramhttp:::py VO sep:::vs:paramhttp:::py VO sep:::vs:paramhttp:::py VO sep:::vs:paramhttp:::py VO sep:::vs:paramhttp:::py VO sep:::vs:paramhttp:::py VO sep:::vs:paramhttp,std:::py VO sep:::std:::py VO sep:::std:::py VO sep:::std:::py VO sep:::std:::py VO sep:::std:::py VO sep:::std:::py VO sep:::std
ivo://bsdc.icranet.org/onebigb/q/cone,vs:catalogservice,1BIGB cone,1BIGB catalog,,"This catalog presents the 1-100 GeV spectral energy distribution\n(SED) for a population of 148 high-synchrotron-peaked blazars (HSPs)\nrecently detected with Fermi-LAT as part of the First Brazil-ICRANet\nGamma-ray Blazar cata

In [23]:
type(services[0])

pyvo.registry.regtap.RegistryResource

In [43]:
result = services[10].service.search(coord, radius * u.deg)

In [46]:
tuple(result)

((10.684542, 41.269194, 0.00023, '720225', '00 42 44.3', '+41 16 09', '2003-10-03T08:49:28', 205, 'o', 'Y', '', 'OBJECT', 'M31', 'MEGAPRIME', 'MEGACAM', 1.144, 'MEGAPRIME', 'BRENT TULLY', 'QSO TEAM', '2005-02-28T10:00:00', -2147483648, -2147483648, '2003BH98', '', 'Archive'),
 (10.680875, 41.268389, 0.00302, '720226', '00 42 43.4', '+41 16 06', '2003-10-03T08:53:47', 205, 'o', 'Y', '', 'OBJECT', 'M31', 'MEGAPRIME', 'MEGACAM', 1.137, 'MEGAPRIME', 'BRENT TULLY', 'QSO TEAM', '2005-02-28T10:00:00', -2147483648, -2147483648, '2003BH98', '', 'Archive'),
 (10.688291, 41.269972, 0.00279, '720228', '00 42 45.2', '+41 16 12', '2003-10-03T09:04:18', 205, 'o', 'Y', '', 'OBJECT', 'M31', 'MEGAPRIME', 'MEGACAM', 1.123, 'MEGAPRIME', 'BRENT TULLY', 'QSO TEAM', '2005-02-28T10:00:00', -2147483648, -2147483648, '2003BH98', '', 'Archive'),
 (10.683458, 41.271944, 0.00305, '720229', '00 42 44.0', '+41 16 19', '2003-10-03T09:08:42', 205, 'o', 'Y', '', 'OBJECT', 'M31', 'MEGAPRIME', 'MEGACAM', 1.117, 'MEGAPRIM

In [56]:
dict(result[0])

{'_RAJ2000': 10.684542,
 '_DEJ2000': 41.269194,
 '_r': 0.00023,
 'expnum': '720225',
 'RAJ2000': '00 42 44.3',
 'DEJ2000': '+41 16 09',
 'ObsDate': '2003-10-03T08:49:28',
 'ExpTime': 205,
 'class': 'o',
 'science': 'Y',
 'preview': '',
 'ObsType': 'OBJECT',
 'Object': 'M31',
 'Instrum': 'MEGAPRIME',
 'Detector': 'MEGACAM',
 'airmass': 1.144,
 'Focus': 'MEGAPRIME',
 'PIname': 'BRENT TULLY',
 'Observer': 'QSO TEAM',
 'RelDate': '2005-02-28T10:00:00',
 'naxis1': -2147483648,
 'naxis2': -2147483648,
 'runID': '2003BH98',
 'Comment': '',
 'CFHT': 'Archive'}

In [57]:
def cone_search(service: vo.registry.regtap.RegistryResource, coord, radius):
    """
    Perform a cone search on a service and return the results as a polars DataFrame
    """
    # Perform the cone search
    result = service.service.search(coord, radius * u.deg)
    
    print(f"Found {len(result)} results from {service.short_name}")

    # If there are no results, return None
    if not result:
        return None

    #Convert the results to a polars DataFrame
    df = pl.DataFrame(data=[dict(row) for row in result])
    
    # Return the results
    return df

# Search each service and return a dictionary of the results
# only collection results until we have over 10000 rows
results = {}
for service in services:
    try:
        df = cone_search(service, coord, radius)
        if df is not None:
            results[service.short_name] = df
    except Exception as e:
        print(e)
        
    if sum([len(df) for df in results.values()]) > 10000:
        break

Found 0 results from LoTSS-DR1 Cone Search
Found 0 results from MSSS Cone Search
Found 0 results from CSIRO ASKAP Cone
Found 0 results from 1BIGB cone
Found 0 results from sds82 cone
Found 0 results from uhecr cone
Found 0 results from wdsdss10 cone
Found 0 results from whsp cone
Found 66 results from B/assocdata
Found 0 results from B/cb
Found 636 results from B/cfht
Found 207 results from B/chandra
Found 253 results from B/dao
Found 0 results from B/gcvs
Found 1326 results from B/gemini
Found 41 results from B/hst
Found 2 results from B/int
Found 24 results from B/iram
Found 919 results from B/jcmt
Found 10 results from B/mk
Found 1 results from B/sn
Found 8 results from B/subaru
Found 553 results from B/swift
Found 6 results from B/vsx
Found 1 results from B/wds
Found 43 results from B/xmm
Found 2 results from I/122
Found 0 results from I/131A
Found 0 results from I/144
Found 1 results from I/146
Found 0 results from I/147
Found 0 results from I/161
Found 0 results from I/170A
Found

In [63]:
results['I/317'].head()

_r,PPMXL,RAJ2000,DEJ2000,pmRA,pmDE,epRA,epDE,e_RAJ2000,e_DEJ2000,e_pmRA,e_pmDE,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,b1mag,b2mag,r1mag,r2mag,imag,Smags,No,fl,2M,NOMAD
f64,str,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i64,i64,str,str
0.094157,"""22117922650944...",10.595733,41.335313,-1.7,-2.0,1990.09,1990.09,72,72,4.0,4.0,14.146,0.025,13.704,0.038,13.705,0.048,15.94,NaN,17.719999,NaN,NaN,"""0-1--""",3,0,"""2M""","""NOMAD"""
0.097707,"""22125434539596...",10.639756,41.360725,2.7,-11.8,1990.41,1990.41,69,69,4.0,4.0,13.527,0.03,12.937,0.037,12.654,0.032,18.1,NaN,17.040001,NaN,16.370001,"""0-1-7""",4,0,"""2M""","""NOMAD"""
0.081731,"""22125430473172...",10.645488,41.345273,-3.1,7.8,1993.11,1993.11,72,72,5.5,5.5,11.768,0.023,11.502,0.021,11.463,0.02,16.93,NaN,NaN,NaN,17.5,"""0---7""",3,0,"""2M""","""NOMAD"""
0.089812,"""22125435206366...",10.661388,41.357135,-7.0,-13.2,1993.11,1993.11,72,72,5.5,5.5,12.788,0.024,12.42,0.022,12.353,0.024,15.65,NaN,NaN,NaN,18.120001,"""0---7""",3,0,"""2M""","""NOMAD"""
0.000078,"""22125365794415...",10.684715,41.269006,-3.5,-7.1,1995.64,1995.67,28,28,1.5,1.5,9.453,0.052,8.668,0.051,8.475,0.051,11.7,11.86,11.63,NaN,10.74,"""-----""",4,2,"""2M""","""NOMAD"""


In [61]:
import os
# Organize the results in folders named after the service
# Save the results to parquet files
# Make sure that the name of the file is valid and the folder exists
if not os.path.exists(f"results/"):
    os.mkdir(f"results/")
for service, df in results.items():
    df.write_parquet(f"results/{service.replace('/', '_')}.parquet")
    


In [62]:
# Check that the files were written
!ls results

B_assocdata.parquet  B_vsx.parquet   I_252.parquet   I_297.parquet
B_cfht.parquet	     B_wds.parquet   I_254.parquet   I_304.parquet
B_chandra.parquet    B_xmm.parquet   I_255.parquet   I_305.parquet
B_dao.parquet	     I_122.parquet   I_261.parquet   I_306A.parquet
B_gemini.parquet     I_146.parquet   I_270.parquet   I_312.parquet
B_hst.parquet	     I_171.parquet   I_271.parquet   I_315.parquet
B_int.parquet	     I_197A.parquet  I_275.parquet   I_317.parquet
B_iram.parquet	     I_207.parquet   I_280B.parquet  I_319.parquet
B_jcmt.parquet	     I_213.parquet   I_284.parquet   I_322A.parquet
B_mk.parquet	     I_220.parquet   I_285.parquet   I_324.parquet
B_sn.parquet	     I_238A.parquet  I_289.parquet
B_subaru.parquet     I_243.parquet   I_290.parquet
B_swift.parquet      I_247.parquet   I_295.parquet
